                                                                                      ETL - ITEMS

- Liberias a usar

In [1]:
import pandas as pd
import numpy as np
import ast
import json
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

ModuleNotFoundError: No module named 'pyarrow'

- Leemos el archivo items

In [ ]:
rows = []
with open (r'C:\Users\Administrador\Documents\HENRY\LAB1\Data\australian_users_items.json', encoding= 'MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))        
df_items = pd.DataFrame(rows)

: 

In [ ]:
df_items

: 

In [ ]:
df_items.info()

: 

- Crear una función lambda para desanidar las filas y agregar las columnas originales

In [ ]:
expand_items = lambda row: pd.DataFrame(row['items']).assign(user_id=row['user_id'], items_count=row['items_count'], steam_id=row['steam_id'], user_url=row['user_url'])

: 

- Aplicamos la función lambda a la columna 'items' y concatenamos los resultados en un nuevo DataFrame


In [ ]:
expanded_items_df = pd.concat(df_items.apply(expand_items, axis=1).tolist(), ignore_index=True)

: 

- Eliminamos columnas redundantes dado que steam_id es igual a user_id

In [ ]:
expanded_items_df.drop(columns=['steam_id', 'user_url'], inplace= True)

: 

- Eliminamos la columna items_count dado que la podemos calcular y la columna item_name la tenemos en el dataframe games

In [ ]:
expanded_items_df.drop(columns=['items_count', 'item_name'], inplace=True)

: 

- Eliminamos otra columna dado que no la necesitamos para el análisis

In [ ]:
expanded_items_df.drop(columns=['playtime_2weeks'], inplace=True)

: 

In [ ]:
expanded_items_df

: 

In [ ]:
expanded_items_df.iloc[:2500]

: 

In [ ]:
expanded_items_df.info()

: 

El archivo es demasaido pesado asi que tuve que hacerlo mas ligero. Utilizizando paquet y eliminamando registros se llego un archivo mas simple. 

- Transformamos en entero la columna playtime_forever

In [ ]:
expanded_items_df['playtime_forever'] = expanded_items_df['playtime_forever'].astype(int)

: 

In [ ]:
sampled_data = expanded_items_df.sample(frac=0.1)


: 

In [ ]:
table = pa.Table.from_pandas(sampled_data)

: 

In [ ]:
pq.write_table(table, 'items1.parquet')

: 

In [ ]:
pq.write_table(table, 'items1.parquet', compression='snappy')

: 

In [ ]:
# Especifica la ruta del archivo Parquet
ruta_archivo_parquet = 'items1.parquet'

# Lee el archivo Parquet en un DataFrame de Pandas
tabla_parquet = pq.read_table(ruta_archivo_parquet)

# Convierte la tabla Parquet en un DataFrame de Pandas
dataframe_parquet1 = tabla_parquet.to_pandas()

# Ahora puedes trabajar con el DataFrame de Pandas como desees
print(dataframe_parquet1.head())


: 

In [ ]:
dataframe_parquet1.info()

: 